In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

# Tool Use


First of all, let's get the available tools from the MCP server.


In [ ]:
from fastmcp.client import Client

mcp_client = Client("http://127.0.0.1:9090/mcp")

async with mcp_client:
    mcp_tools = await mcp_client.list_tools()

In [ ]:
mcp_tools

Now that we have the tools, it's time to translate them into the format that the Groq API expects.


In [45]:
from mcp.types import Tool


def transform_schema_to_parameters(schema: dict) -> dict:
    properties = {}

    for field_name, field_info in schema["properties"].items():
        properties[field_name] = {
            "type": field_info["type"],
            "description": field_info["title"],
        }
        if "default" in field_info:
            properties[field_name]["default"] = field_info["default"]

    return {
        "type": "object",
        "properties": properties,
        "required": schema.get("required"),
    }


def transform_tool_definition(tool: Tool) -> dict:
    return {
        "type": "function",
        "function": {
            "name": tool.name,
            "description": tool.description,
            "parameters": transform_schema_to_parameters(tool.inputSchema),
        },
    }

In [46]:
tools = [
    transform_tool_definition(mcp_tool)
    for mcp_tool in mcp_tools
    if mcp_tool.name != "get_video_clip_from_image"
]

In [ ]:
tools

In [48]:
from groq import Groq

MODEL = "llama-3.3-70b-versatile"
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [49]:
messages = [
    {"role": "system", "content": "You are Kubrick, a video processing assistant."},
    {
        "role": "user",
        "content": "Hey! What's your name?",
    },
]

response = groq_client.chat.completions.create(
    model=MODEL,
    messages=messages,
    stream=False,
    tools=tools,
    tool_choice="auto",
    max_completion_tokens=4096,
)

In [ ]:
response.choices[0].message.content

In [68]:
messages = [
    {
        "role": "system",
        "content": "You are Kubrick, a video processing assistant. The current video name is: ./videos/pass_the_butter_rick_and_morty.mp4",
    },
    {
        "role": "user",
        "content": "I want to get the clip of the video where Rick says 'Pass the butter', please",
    },
]

response = groq_client.chat.completions.create(
    model=MODEL,
    messages=messages,
    stream=False,
    tools=tools,
    tool_choice="auto",
    max_completion_tokens=4096,
)

In [69]:
response.choices[0].message.content

In [70]:
tool_calls = response.choices[0].message.tool_calls

In [ ]:
tool_calls[0].function.name

In [72]:
import json

inputs = json.loads(tool_calls[0].function.arguments)

In [ ]:
inputs

In [ ]:
async with mcp_client:
    inputs = {
        "user_query": "Rick says Pass the butter",
        "video_path": "./videos/pass_the_butter_rick_and_morty.mp4",
    }
    result = await mcp_client.call_tool("get_video_clip_from_user_query", inputs)
    print(result)

In [ ]:
print(result[0].text)

In [ ]:
import json

json.loads(result[0].text)

In [22]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

In [23]:
tool_calls